In [1]:
!pip install pyspark==3.0.0 koalas==1.0.1 pyarrow==0.15.1 scikit-learn==0.23.1

     |████████████████████████████████| 204.7MB 136kB/s eta 0:00:01   |█                               | 6.3MB 7.2MB/s eta 0:00:28     |█▊                              | 10.7MB 7.2MB/s eta 0:00:27     |███████████▏                    | 71.5MB 14.1MB/s eta 0:00:10     |█████████████████████████       | 160.6MB 9.0MB/s eta 0:00:05     |███████████████████████████████ | 198.6MB 8.8MB/s eta 0:00:01
     |████████████████████████████████| 696kB 29.7MB/s eta 0:00:01
     |████████████████████████████████| 59.2MB 36.6MB/s eta 0:00:01
     |████████████████████████████████| 6.9MB 8.3MB/s eta 0:00:01
     |████████████████████████████████| 204kB 29.5MB/s eta 0:00:01
     |████████████████████████████████| 307kB 30.2MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
ERROR: autoai-libs 1.10.5 has requirement numpy>=1.16.4, but you'll have numpy 1.15.4 which is incompatible.
ERROR: autoai-li

In [14]:
%%bash
for i in `seq 4`; do 
    rm log$i*
    wget https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/log$i.txt
done

--2020-07-03 05:36:16--  https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/log1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19954 (19K) [text/plain]
Saving to: ‘log1.txt’

     0K .......... .........                                  100% 13.0M=0.001s

2020-07-03 05:36:16 (13.0 MB/s) - ‘log1.txt’ saved [19954/19954]

--2020-07-03 05:36:16--  https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/log2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13026 (13K) [text/plain]
Saving to: ‘log2.txt’

     0K .......... ..      

In [35]:
%%bash
for i in `seq 4`; do 
    echo "ts,rssi" > log$i.csv
    cat log$i.txt |awk  '{print $4","$11}' >> log$i.csv
done

In [17]:
%%bash
for i in `seq 4`; do 
    rm markers$i.csv
    wget https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/markers$i.csv
done

--2020-07-03 05:38:36--  https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/markers1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151 [text/plain]
Saving to: ‘markers1.csv’

     0K                                                       100% 4.71M=0s

2020-07-03 05:38:36 (4.71 MB/s) - ‘markers1.csv’ saved [151/151]

--2020-07-03 05:38:36--  https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/markers2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151 [text/plain]
Saving to: ‘markers2.csv’

     0K                        

In [19]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.getOrCreate()

In [101]:
from pyspark.sql.functions import lit

data = None
markers = None

for i in range(1,5):
    data_tmp = spark.read.option('header','true').csv('log'+str(i)+'.csv')
    data_tmp = data_tmp.withColumn('receiver_placement', lit(i))
    markers_tmp = spark.read.option('header','true').csv('markers'+str(i)+'.csv')
    markers_tmp = markers_tmp.withColumn('receiver_placement', lit(i))
    if i == 1:
        data = data_tmp
        markers = markers_tmp
    else:
        data = data.union(data_tmp)
        markers = markers.union(markers_tmp)

[Row(receiver_placement=1)]
[Row(receiver_placement=1), Row(receiver_placement=2)]
[Row(receiver_placement=1), Row(receiver_placement=3), Row(receiver_placement=2)]
[Row(receiver_placement=1), Row(receiver_placement=3), Row(receiver_placement=4), Row(receiver_placement=2)]


In [103]:
#convert ts_string to ts_timestamp
from pyspark.sql.functions import to_timestamp
data = data.withColumn("ts_p", to_timestamp("ts", "HH:mm:ss"))
markers = markers.withColumn("ts_start_p", to_timestamp("ts_start", "HH:mm:ss"))
markers = markers.withColumn("ts_stop_p", to_timestamp("ts_stop", "HH:mm:ss"))

In [104]:
data.createOrReplaceTempView('data')
markers.createOrReplaceTempView('markers')

In [105]:
data = spark.sql('''
select rssi,d.receiver_placement,location from data d 
inner join markers m on 
d.receiver_placement = m.receiver_placement 
and d.ts_p>=m.ts_start_p 
and d.ts_p<=m.ts_stop_p
''')
data.createOrReplaceTempView('data')

In [106]:
import databricks.koalas as ks
kdf = data.to_koalas()

In [107]:
kdf = kdf.dropna()

In [108]:
pdf = kdf.toPandas()

In [109]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(pdf[['rssi','receiver_placement']], pdf[['location']])

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [110]:
from sklearn.metrics import accuracy_score
accuracy_score(pdf[['location']], clf.predict(pdf[['rssi','receiver_placement']]))

0.27932960893854747

In [111]:
from sklearn.metrics import classification_report
target_names = ['0','1','2','3']
print(classification_report(pdf[['location']], clf.predict(pdf[['rssi','receiver_placement']]), target_names=target_names))

              precision    recall  f1-score   support

           0       0.36      0.67      0.47        92
           1       0.00      0.00      0.00        88
           2       0.00      0.00      0.00        81
           3       0.24      0.39      0.30        97

    accuracy                           0.28       358
   macro avg       0.15      0.27      0.19       358
weighted avg       0.16      0.28      0.20       358



/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [165]:
slices = [[x]*4 for x in [b for b in range(4)]]

for receiver_placement in range(1,5):
    for location in range(4):
        slices[location][receiver_placement-1] = kdf[(kdf['receiver_placement']==receiver_placement) & (kdf['location']==location)][['rssi','location']]


In [166]:
minimum = 100
for i in range(4):
    for j in range(4):
        minimum = min(minimum,len(slices[i][j]))

In [167]:
kdf_final = ks.range(minimum*4)
for i in range(4): #locations
    for j in range(4): #placements
        slices[i][j] = slices[i][j].head(minimum)


In [174]:
ks.options.compute.ops_on_diff_frames=True
for i in range(4): #locations
    kdf_tmp = ks.range(minimum)
    for j in range(4): #placements
        if j == 0:
             kdf_tmp['location']=slices[i][j]['location']
        kdf_tmp['rssi_'+str(j)]=slices[i][j]['rssi']
    print(kdf_tmp)  

   id location rssi_0 rssi_1 rssi_2 rssi_3
0   0        0    -86   None   None   None
1   1        0    -84   None   None   None
2   2        0    -86   None   None   None
3   3        0    -87   None   None   None
4   4        0    -77   None   None   None
5   5        0    -72   None   None   None
6   6        0    -66   None   None   None
7   7        0    -73   None   None   None
8   8        0    -67   None   None   None
   id location rssi_0 rssi_1 rssi_2 rssi_3
0   0     None   None   None   None   None
1   1     None   None   None   None   None
2   2     None   None   None   None   None
3   3     None   None   None   None   None
4   4     None   None   None   None   None
5   5     None   None   None   None   None
6   6     None   None   None   None   None
7   7     None   None   None   None   None
8   8     None   None   None   None   None
   id location rssi_0 rssi_1 rssi_2 rssi_3
0   0     None   None   None   None   None
1   1     None   None   None   None   None
2   2     N

In [145]:
ks.DataFrame({},index=list(['rssi_1','rssi_2','rssi_3','rssi_4','location']))

""
rssi_1
rssi_2
rssi_3
rssi_4
location


,id
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9
